In [0]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import pandas as pd
from bs4 import SoupStrainer
import pandas as pd
import time

#use requests to get the html of the site
r = requests.get('https://www.socialseer.com/resources/us-senator-twitter-accounts/')
#beautifulsoup formats as needed
soup = BeautifulSoup(r.text, 'html.parser')

In [0]:
soup
table = pd.read_html(r.text)
needed = table[0]
df = pd.DataFrame(needed)

In [0]:
df.columns = df.iloc[0]
test = df.reindex(df.index.drop(0))



In [0]:
twitter_page = []

for stuff, handle in test.iterrows():
  twitter_page.append(f'https://twitter.com/{handle[2]}')

In [0]:
test['Twitter_Page'] = twitter_page

In [0]:
#test.to_csv('senators.csv')
test

,State,Senator,Official Twitter,Staff Twitter,Campaign Twitter,Twitter_Page
1,Alabama,"Shelby, Richard",SenShelby,NaN,NaN,https://twitter.com/SenShelby
2,Alabama,"Jomes, Doug",SenDougJones,NaN,NaN,https://twitter.com/SenDougJones
3,Alaska,"Murkowski, Lisa",lisamurkowski,NaN,LisaForSenate,https://twitter.com/lisamurkowski
4,Alaska,"Sullivan, Dan",SenDanSullivan,NaN,DanSullivan2014,https://twitter.com/SenDanSullivan
5,Arizona,"Kyl, Jon",SenJonKyl,NaN,NaN,https://twitter.com/SenJonKyl
6,Arizona,"Flake, Jeff",JeffFlake,NaN,FlakeforSenate,https://twitter.com/JeffFlake
7,Arkansas,"Cotton, Tom",SenTomCotton,NaN,TomCottonAR,https://twitter.com/SenTomCotton
8,Arkansas,"Boozman, John",JohnBoozman,NaN,Boozman4AR,https://twitter.com/JohnBoozman
9,California,"Feinstein, Dianne",SenFeinstein,NaN,DianneFeinstein,https://twitter.com/SenFeinstein
10,California,"Harris, Kamala",SenKamalaHarris,NaN,KamalaHarris,https://twitter.com/SenKamalaHarris


In [0]:
import tweepy #https://github.com/tweepy/tweepy
import csv

#Twitter API credentials
consumer_key = "nQyetU704Vy1Kgv6yUwkAq3b5"
consumer_secret = "MV2Eg6meTcqH7oRWKbFnGVbjNmAYTzCz9Dpyl3nvs8auGB853F"
access_key = "168749635-Je8cGygyVSdvG3RsyqpLa89rFd7fs9q0juK1lOXr"
access_secret = "2oO0GAAdD8fA9QwtHWCyn5prE0fqAbsTzGLxTSMsb2Bei"



def get_all_tweets(screen_name):
	#Twitter only allows access to a users most recent 3240 tweets with this method
	
	#authorize twitter, initialize tweepy
	auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
	auth.set_access_token(access_key, access_secret)
	api = tweepy.API(auth)
	
	#initialize a list to hold all the tweepy Tweets
	alltweets = []	
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
	new_tweets = api.user_timeline(screen_name = screen_name,count=200)
	
	#save most recent tweets
	alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
	oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no tweets left to grab
	while len(new_tweets) > 0:
		print ("getting tweets before %s" % (oldest))
		
		#all subsiquent requests use the max_id param to prevent duplicates
		new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
		
		#save most recent tweets
		alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
		oldest = alltweets[-1].id - 1
		
		print ("...%s tweets downloaded so far" % (len(alltweets)))
	
	#transform the tweepy tweets into a 2D array that will populate the csv	
	outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
	
	#write the csv	
	with open('%s_tweets.csv' % screen_name, 'w') as f:
		writer = csv.writer(f)
		writer.writerow(["id","created_at","text"])
		writer.writerows(outtweets)
	
	pass



In [0]:
from urllib.error import URLError, HTTPError
from urllib.request import urlopen
# for data, item in test.iterrows():
#   try:
#     print(item[5])
#     print(urllib.request.urlopen(f"{item[5]}").getcode())
#   except HTTPError:
#     print(f"{item[1]} is incorrect")
  
  


In [0]:
#need to re-upload the senator.csv file due to Michael Bennet
senators_list = pd.read_csv('senators.csv')
for data, item in senators_list.iterrows():
  try:
    print(item[6])
    print(urllib.request.urlopen(f"{item[6]}").getcode())
  except HTTPError:
    print(f"{item[2]} is incorrect")


https://twitter.com/SenShelby
200
https://twitter.com/SenDougJones
200
https://twitter.com/lisamurkowski
200
https://twitter.com/SenDanSullivan
200
https://twitter.com/SenJonKyl
200
https://twitter.com/JeffFlake
200
https://twitter.com/SenTomCotton
200
https://twitter.com/JohnBoozman
200
https://twitter.com/SenFeinstein
200
https://twitter.com/SenKamalaHarris
200
https://twitter.com/SenatorBennet
200
https://twitter.com/SenCoryGardner
200
https://twitter.com/ChrisMurphyCT
200
https://twitter.com/SenBlumenthal
200
https://twitter.com/SenatorCarper
200
https://twitter.com/ChrisCoons
200
https://twitter.com/SenBillNelson
200
https://twitter.com/marcorubio
200
https://twitter.com/SenDavidPerdue
200
https://twitter.com/SenatorIsakson
200
https://twitter.com/brianschatz
200
https://twitter.com/maziehirono
200
https://twitter.com/MikeCrapo
200
https://twitter.com/SenatorRisch
200
https://twitter.com/SenDuckworth
200
https://twitter.com/SenatorDurbin
200
https://twitter.com/SenDonnelly
200
htt

In [0]:
senators_list
for header in senators_list:
  time.sleep(5)
  print(header)

Unnamed: 0
State
Senator
Official Twitter
Staff Twitter
Campaign Twitter
Twitter_Page


In [0]:
for data, username in senators_list.iterrows():
  time.sleep(65)
  print ("Start Time : %s" % time.ctime())
  print(username[2])
  get_all_tweets(username[3])
  print ("End Time: %s" % time.ctime())

Start Time : Fri Apr 12 00:56:11 2019
Shelby, Richard
getting tweets before 1060273822346235904
...400 tweets downloaded so far
getting tweets before 1004839920589070337
...600 tweets downloaded so far
getting tweets before 943484607089037311
...800 tweets downloaded so far
getting tweets before 845725514400743425
...999 tweets downloaded so far
getting tweets before 809490542585401343
...1198 tweets downloaded so far
getting tweets before 747429376099160063
...1398 tweets downloaded so far
getting tweets before 699619718311718913
...1598 tweets downloaded so far
getting tweets before 643852236972556293
...1798 tweets downloaded so far
getting tweets before 568065838554746879
...1998 tweets downloaded so far
getting tweets before 382900848092643327
...2149 tweets downloaded so far
getting tweets before 63640521595301887
...2149 tweets downloaded so far
End Time: Fri Apr 12 00:56:16 2019
Start Time : Fri Apr 12 00:57:21 2019
Jomes, Doug
getting tweets before 1063152269758287873
...400 t

In [0]:
get_all_tweets('sensanders')

getting tweets before 1091063412807921663
...400 tweets downloaded so far
getting tweets before 1074792367142641663
...600 tweets downloaded so far
getting tweets before 1062715768524210175
...800 tweets downloaded so far
getting tweets before 1042522322421379071
...1000 tweets downloaded so far
getting tweets before 1023946765907750911
...1200 tweets downloaded so far
getting tweets before 1011320334946103295
...1400 tweets downloaded so far
getting tweets before 994580836753248255
...1600 tweets downloaded so far
getting tweets before 976249923443331072
...1800 tweets downloaded so far
getting tweets before 963514254207934465
...2000 tweets downloaded so far
getting tweets before 955589263030210561
...2200 tweets downloaded so far
getting tweets before 942781642447511552
...2400 tweets downloaded so far
getting tweets before 928674167859408896
...2600 tweets downloaded so far
getting tweets before 914500284033978368
...2799 tweets downloaded so far
getting tweets before 9080308474203

In [0]:
billcassidy = pd.read_csv("BillCassidy_tweets.csv")

In [0]:
billcassidy
almost_tweets =[]
closer_tweets = []
test_tweets = []

for data, tweet in billcassidy.iterrows():
  tweets = tweet[2]
  almost_tweets.append(tweets)
  
for sayings in almost_tweets:
  closer_tweets.append(sayings.replace('\\xe2\\x80\\xa6', ''))
  
for rackets in closer_tweets:
    test_tweets.append(rackets.replace('\\xe2\\x80\\x99', ""))


  
test_tweets
  

["b'March Madness bracket on fire? You can cool down your drink at least with one of our new koozies. Chip in just $7 t https://t.co/s0tlU4gTXo'",
 "b'Thrilled to see that @CenturyLink, one of northern Louisianas top employers, has extended its stay in Monroe!\\n\\nhttps://t.co/ZWLo41SoH8'",
 "b'How far will the Left go to risk the lives of unborn babies? Theyve had 29 chances to do the right thing. Enough i https://t.co/wKQlxc63Sr'",
 "b'Its (spring) game day! @LSUfootball is back!\\n\\nhttps://t.co/0nFIYe1NUC'",
 "b'This Sunday is World Health Day. I am proud to have dedicated my life to helping patients. I am even more proud to https://t.co/3waupgZO8Y'",
 'b"196k jobs added in March confirms that President @realDonaldTrump\'s pro-growth economic policies are working to ben https://t.co/YgvMv8kQt7"',
 "b'#ICYMI Our Louisiana ports are growing and investing millions into our coastal communities. Read about our surging https://t.co/csiCIYuB8V'",
 "b'Make no mistake: there is absolutely

In [0]:
import numpy as np

In [0]:
# with open("imdb_labelled.txt", "r") as text_file:
#   try:
#     lines = text_file.read().split("\n")
#   except UnicodeDecodeError:
#     pass
# with open("amazon_cells_labelled.txt", "r") as text_file:
#   try:
#     lines = text_file.read().split("\n")
#   except UnicodeDecodeError:
#     pass
# with open("yelp_labelled.txt", "r") as text_file:
#   try:
#     lines = text_file.read().split("\n")
#   except UnicodeDecodeError:
#     pass
# newLines = [line.split("\t") for line in lines if len(line.split("t")) == 2 and line.split("\t")[1] != ""]

In [0]:
def loadCSV(filePath):
    dataframe = pd.read_csv(filePath, encoding='iso-8859-1')
    dataframe.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
    #datakeys = dataframe.keys();
    return dataframe

tweet_training_file = loadCSV('testdata.manual.2009.06.14.csv')
type(tweet_training_file)

pandas.core.frame.DataFrame

In [0]:
for data, information in tweet_training_file.iterrows():
  print(information)

target                                                    4
id                                                        4
date                           Mon May 11 03:18:03 UTC 2009
flag                                                kindle2
user                                                 vcu451
text      Reading my kindle2...  Love it... Lee childs i...
Name: 0, dtype: object
target                                                    4
id                                                        5
date                           Mon May 11 03:18:54 UTC 2009
flag                                                kindle2
user                                                 chadfu
text      Ok, first assesment of the #kindle2 ...it fuck...
Name: 1, dtype: object
target                                                    4
id                                                        6
date                           Mon May 11 03:19:04 UTC 2009
flag                                                ki

In [0]:
train_documents = [line[5] for data, line in tweet_training_file.iterrows()]
train_labels = [int(line[0]) for data, line in tweet_training_file.iterrows()]

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB

In [0]:
count_vectorizer = CountVectorizer(binary="false")
train_documents = count_vectorizer.fit_transform(train_documents)

In [0]:
classifier = BernoulliNB().fit(train_documents, train_labels)

In [0]:
def predictionOutput(sentence):
    prediction = classifier.predict(count_vectorizer.transform([sentence]))
    if(prediction[0] == 4):
        print("This is a Positive Sentiment Sentence")
    elif (prediction[0] == 2):
        print("This is a Neutral Sentiment Sentence")    
    elif (prediction[0] == 0):
        print("This is a Negative Sentiment Sentence")

In [0]:
for prediction in test_tweets:
  print(prediction)
  predictionOutput(prediction)

b'March Madness bracket on fire? You can cool down your drink at least with one of our new koozies. Chip in just $7 t https://t.co/s0tlU4gTXo'
This is a Positive Sentiment Sentence
b'Thrilled to see that @CenturyLink, one of northern Louisianas top employers, has extended its stay in Monroe!\n\nhttps://t.co/ZWLo41SoH8'
This is a Positive Sentiment Sentence
b'How far will the Left go to risk the lives of unborn babies? Theyve had 29 chances to do the right thing. Enough i https://t.co/wKQlxc63Sr'
This is a Negative Sentiment Sentence
b'Its (spring) game day! @LSUfootball is back!\n\nhttps://t.co/0nFIYe1NUC'
This is a Negative Sentiment Sentence
b'This Sunday is World Health Day. I am proud to have dedicated my life to helping patients. I am even more proud to https://t.co/3waupgZO8Y'
This is a Negative Sentiment Sentence
b"196k jobs added in March confirms that President @realDonaldTrump's pro-growth economic policies are working to ben https://t.co/YgvMv8kQt7"
This is a Positive Sentim